<h1><center>Laboratorio 9: Los huesos de Hipócrates 🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Andrés Gonzalez Fuentes
- Nombre de alumno 2: Camilo Carvajal Reyes

### **Link de repositorio de GitHub:** `https://github.com/camilocarvajalreyes/laboratorios-MDS7202`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

## Temas a tratar

- Creación de clasificadores de imagenes a traves de redes Fully connected y CNN.
- Uso de Dataloaders para la carga de datasets.
- Comparación de Fully Connected y red convolucional.

## Reglas:

- Fecha de entrega: 17/11/2022
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Creación de modelos de clasificación de imágenes utilizando Pytorch.
- Creación de dataloader y aplicar transformaciones sobre el dataset.
- Comprender la diferencia entre una CNN y una Fully Connected.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones de `Pytorch`, la cual, está enfocada para proyectos de Deep Learning.


# Importamos librerias utiles 😸

Comenzamos importando librerías utiles para la ejecución del laboratorio:

In [1]:
import os
import time
from PIL import Image

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import datasets, models
from torchvision import transforms as T

# Identificando los Huesos de Hipócrates🔎

<p align="center">
  <img src="https://i.pinimg.com/originals/d8/58/66/d85866cd1cc3979f94526551addf74b4.gif" width="300">
</p>

Tras el éxito que han tenido proponiendo modelos de machine learning en trabajos anteriores, el famoso medico Hipócrates se ha contactado con ustedes para solicitarles ayuda para automatizar la identificación de radiografías de partes humanas. Para esto, les señala que le gustaría utilizar algoritmos de deep learning producto que Demócrito le señalo que resultan la mejor alternativa para la predicción de imágenes.

En su conversación con el medico usted le comenta que ha tenido algunas clases relacionadas a Deep Learning, por esto, están motivados en abordar el problema utilizando redes Fully Connected y redes convolucionales con Pytorch. Sin embargo, al anunciarle los tipos de redes que conocen, el filósofo les comenta que no había escuchado muy buenos resultados por parte de las CNN, por lo que les pide que comprueben a traves de la métrica de accuracy que tipo de redes es mejor para la tarea de identificación de radiografías. ¿Será cierto lo que dice el filósofo?, Veámoslo en un nuevo capítulo de los Laboratorios de Programación Científica para Ciencia de Datos!

## 1.1 Creación de Lista de Archivos

<p align="center">
  <img src="https://media.tenor.com/BJ-9w-MUVCMAAAAM/tis100-sad.gif" width="300">
</p>


Comience revisando de forma manual cada una de las imágenes que posee la carpeta subida a material docente. Verifique la cantidad de tipos de radiografías que se tienen y la cantidad de imágenes que dispone cada carpeta.

Revisado el contenido de las imágenes, utilice `os.listdir` para crear un `numpy.array` o un `Dataframe` que contenga las imágenes y un label que señale al tipo de radiografía a la que hace referencia la imagen. Para hacer las etiquetas codifique el tipo de imágenes en números que vayan del 0 al total de tipos de radiografías, no utilice strings para codificar las etiquetas.

**Ejemplo de Estructura:**

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax"></th>
    <th class="tg-0lax">image_path</th>
    <th class="tg-0lax">label</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">image1</td>
    <td class="tg-0lax">1</td>
  </tr>
  <tr>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">image2</td>
    <td class="tg-0lax">0</td>
  </tr>
  <tr>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">image3</td>
    <td class="tg-0lax">2</td>
  </tr>
  <tr>
    <td class="tg-0lax">3</td>
    <td class="tg-0lax">image4</td>
    <td class="tg-0lax">0</td>
  </tr>
  <tr>
    <td class="tg-0lax">4</td>
    <td class="tg-0lax">image5</td>
    <td class="tg-0lax">4</td>
  </tr>
</tbody>
</table>

In [2]:
X = []
y = []
for i, category in enumerate(os.listdir("Medical-MNIST")):
    for image in os.listdir(f"Medical-MNIST/{category}"):
        X.append(f"{category}/{image}")
        y.append(i)

xray_df = pd.DataFrame({'image_path':X, 'label':y})
xray_df.head()

,image_path,label
0,AbdomenCT/001857.jpeg,0
1,AbdomenCT/007792.jpeg,0
2,AbdomenCT/007577.jpeg,0
3,AbdomenCT/002785.jpeg,0
4,AbdomenCT/005169.jpeg,0


## 1.2 Creación de Dataset

Tomando en cuenta la estructura de datos desarrollada en el punto 1.1, construya la clase `MedicalDataset()` que cumpla los siguientes puntos:

- [X] Poseer un `__init__` en el que se almacene `estructura` creada en 1.1, la `raiz` de la carpeta y una función que permita transformar el dataset (de esto no se preocupe mucho, ya que solamente debe almacenar una función en el atributo).
- [X] La clase debe ser capaz de entregar la cantidad de elementos a traves de `__len__`.
- [X] Debe poseer el método `__getitem__` que retorne una tupla con la imagen y su correspondiente etiqueta.

In [3]:
# Código Aquí
class MedicalDataset(Dataset):
    def __init__(self, estructura, raiz, transform):
        self.estructura = estructura
        self.raiz = raiz
        self.transform = transform
    
    def __getitem__(self, idx):
        
        # Un poco de ayuda para cargar la imagen
        img_path = f"{self.raiz}/{self.estructura.image_path[idx]}"
        image = Image.open(img_path).convert('RGB')
        label = self.estructura.label[idx]
        
        # Auida para realizar la transformación
        if self.transform:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return self.estructura.shape[0]

## 1.3 Prueba del MedicalDataset

Con la clase construida en el punto 1.2, verifique su funcionamiento cargando el dataset y realizando las transformaciones que entrega la función `transform_image`. Compruebe a través de un ejemplo las transformaciones aplicadas en la imagen, comentando la función que cumple `MedicalDataset` y si es posible observar todas las transformaciones aplicadas con la función `transform_image`.

- [X] Probar la clase MedicalDataset y aplicando una transformación de "train"
- [X] Plotear un ejemplo del MedicalDataset.

**Función para transformar las imagenes:**

In [4]:
dim = 64

def transform_image(stage = None):
    
    if stage == "train":
        # Tr_img = T.Compose([T.Resize(size = (256,256)),
        Tr_img = T.Compose([T.Resize(size = (dim,dim)),
                T.RandomRotation(degrees = (-20,+20)),
                T.ToTensor()])
        
    elif stage == "test" or stage == "val":
        # Tr_img = T.Compose([T.Resize(size = (224,224)), T.ToTensor()]) 
        Tr_img = T.Compose([T.Resize(size = (dim,dim)), T.ToTensor()]) 

    return Tr_img

**Código para obtener un ejemplo:**

In [5]:
# Prueba del dataset
dataset = MedicalDataset(xray_df, "Medical-MNIST", transform_image("train"))

In [6]:
data_test= iter(dataset)
example_image = next(data_test)

# Utilice plotly para plotear un ejemplo
px.imshow(example_image[0][0], binary_string=True)

> Comente que realiza la clase construida y las transformaciones aplicadas.
```
La clase construida se encarga de crear un dataset que luego pueda ser implementado en conjunto con un Dataloader de Pytorch, para lo que tiene que contar con los métodos '__getitem__' y '__len__' para entregar un elemento del dataset iterativamente según un índice y para contar con el largo de todo el dataset. Luego, las transformaciones aplicadas sirven para reajustar una imagen a un tamaño estándar de 256x256 y luego una pequeña rotación random, seguramente para entrenar un modelo que luego sea robusto a las rotaciones. Finalmente se hace la transformación a un tensor de Pytorch, para que pueda ser utilizado en un modelo.
```

## 1.4 Creación de Clasificadores

<p align="center">
  <img src="https://149695847.v2.pressablecdn.com/wp-content/uploads/2018/01/conv-full-layer.gif" width="300">
</p>

A continuación, deben construir tres clasificadores con los que deberán verificar cuál de las arquitecturas posee un mejor desempeño para la tarea de clasificación de imágenes. Para la construcción considere los siguientes puntos:

- [X] Señale cual es el objetivo del `forward` en este tipo de redes, sea breve para su explicación.
- [X] Construir una red Fully Connected para solucionar el problema de clasificación. Para esta parte se le aconseja que rellene el esqueleto dispuesto más abajo y que lleva el nombre de `FCClassifier`, en el deberá rellenar con la dimensión de las capas ocultas y verificar cual será el tamaño de la entrada.
- [X] Construya una red convolucional **simple** (no más de una capa convolucional) para la tarea de clasificación de imágenes, para esto basen su código en la clase del día `09-11-2022`.
- [X] Crear una red convolucional más compleja. Para esta parte tienen completa libertad en la construcción de su red, lo único que debe cumplir es que sea convolucional.

**Esqueletos Propuestos:**

In [7]:
class FCClassifier(nn.Module):
    def __init__(self, in_dimension, num_classes):
        super(FCClassifier, self).__init__()
        self.flatten = nn.Flatten()
        self.lin1 = nn.Linear(in_dimension, 64)
        self.lin2 = nn.Linear(64, num_classes)
        
    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.lin1(x))
        return self.lin2(x)

In [8]:
class CNNClassifier1(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNNClassifier1, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            )
        self.fc = nn.Linear(in_features=32 * 128 * 128, out_features=num_classes)
    
    def forward(self, x):
        x = self.layer1(x)
        return self.fc(x.flatten())

    
class CNNClassifier2(nn.Module):
    def __init__(self, channel_list, num_classes):
        super(CNNClassifier2, self).__init__()
        self.layers = nn.ModuleList()
        for idx in range(1, len(channel_list)):
            if idx % 2 ==0:
                layer = nn.Sequential(
                    nn.Conv2d(in_channels=channel_list[idx-1], out_channels=channel_list[idx], kernel_size=3, stride=1, padding=1),
                    nn.BatchNorm2d(channel_list[idx]),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size = 2, stride = 2))
            else:
                layer = nn.Sequential(
                    nn.Conv2d(in_channels=channel_list[idx-1], out_channels=channel_list[idx], kernel_size=3, stride=1, padding=1),
                    nn.BatchNorm2d(channel_list[idx]),
                    nn.ReLU())
            self.layers.append(layer)

        self.fc1 = nn.Sequential(
                    nn.Dropout(0.5),
                    nn.Linear(64 * 64 * channel_list[-1], 1024),
                    nn.ReLU())
        self.fc2 = nn.Sequential(
                    nn.Dropout(0.5),
                    nn.Linear(1024, 1024),
                    nn.ReLU())                    
        self.fc3 = nn.Linear(in_features=1024, out_features=num_classes)
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.fc1(x.flatten())
        x = self.fc2(x)
        return self.fc3(x)

> Respuesta
```
El objetivo de el método 'forward' es hacer el cálculo de las salidas de cada capa dentro de un modelo a partir de sus entradas, realizando un paso de los gradientes en aquellos tensores entrenables para posteriormente propagar los errores con backpropagation. 
```

## 1.5 Separando Datos para el Entrenamiento

<p align="center">
  <img src="https://c.tenor.com/Esn7Jif-ZLQAAAAC/separate-square.gif" width="200">
</p>

Utilizando un Holdout a su gusto, separe los datos en un conjunto de entrenamiento y de testing. Aplique las transformaciones correspondientes usando `transform_image` para cada conjunto de datos y utilice `torch.utils.data.DataLoader` para crear un objeto iterable del dataset.

- [X] Definir el Holdout a utilizar.
- [X] Separar los datos en un conjunto de entrenamiento y prueba.
- [X] Aplicar las transformaciones correspondientes en cada uno de los dataset.
- [X] Utilizar `DataLoader` de pytorch sobre los dataset.

In [9]:
# Separar Datos de Entrenamiento
from sklearn.model_selection import train_test_split

test_size = 0.3
X_train, X_test = train_test_split(xray_df, test_size=test_size, random_state=42)
X_train = X_train.reset_index()
X_test = X_test.reset_index()
X_train = MedicalDataset(X_train, "Medical-MNIST", transform_image("train"))
X_test = MedicalDataset(X_test, "Medical-MNIST", transform_image("test"))

In [10]:
# Uso de torch.utils.data.DataLoader
batch_size = 16

train_loader = DataLoader(
    X_train,
    batch_size = batch_size,
    pin_memory = True,
    num_workers = 12,
    shuffle = True
    )

test_loader = DataLoader(
    X_test,
    batch_size = batch_size,
    pin_memory = True,
    num_workers = 0,
    shuffle = False
    )

## 1.6 Creación de Funciones de Entrenamiento y Evaluación

<p align="center">
  <img src="https://www.researchgate.net/publication/319535615/figure/fig3/AS:536187598065664@1504848493070/A-typical-convolutional-neural-network-CNN-Architecture-for-Medical-Image-Classification.png" width="500">
</p>


Ya construido todas las funciones y clases necesarias llego el momento más importante... probar la red. Para esta sección, ustedes deberán ser capaces de definir los hiperparámetros de la red, definir las funciones de perdida a utilizar, señalar el optimizador a usar y finalmente crear sus funciones para el entrenamiento y prueba. Para realizar esta parte más estructurada, seguir los siguientes puntos de forma secuencial:

- [ ] Especifique los Hiperparámetros de las 3 redes. Para esta parte sea claro de su elección y señale el porqué de sus elecciones (o sea justifique el setting de sus hiperparámetros).
- [x] Defina los modelos a utilizar, el optimizador que utilizará para el modelo y señale la función de perdida que utilizará.
- [x] Explique de forma breve la función que cumplen los pasos `Backward` y `Descenso del gradiente` en una red neuronal.
- [x] Cree una función llamado `train` que entrene a los clasificadores. Para esto, recuerde que estos modelos suelen utilizar un número de épocas, por lo que deberá generar un proceso iterativo de entrenamiento. Es importante que su función imprima las `loss` obtenidas por el modelo en cada época (si gusta puede almacenar estas losses en una lista para luego graficarlas y comparar).
- [x] Diseñe una función para evaluar el desempeño de las redes. Para evaluar las redes utilice solamente la métrica accuracy (para esto se le recomienda comparar la predicción con el ground truth)

In [11]:
# Especificar hyperparámetros de las redes
# dim = 64  # dimensión del reshape al pre-procesar los datos

In [12]:
num_classes = 6
in_dimension = dim * dim * 3
# Red 1
model_fc = FCClassifier(in_dimension, num_classes)

In [13]:
in_channels_cnn = 3
# Red 2
model_CNN1 = CNNClassifier1(in_channels_cnn, num_classes)

In [14]:
channels_list_cnn2 = [3,dim,dim,128,128,256]
# Red 3
model_CNN2 = CNNClassifier2(channels_list_cnn2, num_classes)

In [15]:
modelos = [model_fc, model_CNN1, model_CNN2]
# modelos = [model_fc]

In [16]:
# parámetros de entrenamiento
lr = 1e-4
n_epochs = 5  # 20

```
Explicación parámetros
```

In [17]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam()

```
Para el criterio elegimos la pérdida de entropía cruzada. Esta corresponde a la elección natural en un problema de clasificación multimodal y equivale a la minimización del estimador de máxima verosimilitud.

Por otro lado, escogemos el optimizador Adam, que es una variación de SGD ampliamente utilizado y que combina elementos de AdaGrad y RMSProp.
```

Nota: redefiniremos el optimizador dentro de la función entrenar. Esto pues el optimizador guarda parámetros de cada modelo en particular.

```
Backward:
Durante el entrenamiento, una vez que la información pasa a través de la red neuronal para computar una predicción, se computará un error (forward). Enseguida, se computa
el gradiente usando las operaciones que estaban guardadas y mediante el uso exhaustivo de la regla de la cadena, haciendo fluir la información en reversa hasta tener el valor de cada derivada parcial (backward).


Descenso del gradiente (estocástico):
Esta parte de las iteraciones aplica el paso del algoritmo descenso de gradiente, o como en este caso, una variación de este. Para esto, toma el gradiente calculado en el paso backward y los resta a los valores de los parámetros de nuestra red escalado por el learning rate. El paso posterior .zero_grad() "limpia" el optimizador de los errores de pasos anteriores.
```

In [18]:
# train
def train(model, train_loader, num_epochs, verbose=True):

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    perdidas = []
    
    for e in range(num_epochs):
        perdidas_epoch = []
        for i, (data, label) in enumerate(train_loader):
            #Forward
            pred = model(data)
            
            #Backward
            loss = criterion(pred,label)
            loss.backward()
            
            # Descenso del gradiente
            optimizer.step()
            optimizer.zero_grad()

            perdidas_epoch.append(loss.item())

        perdidas += perdidas_epoch

        if verbose:
            print("epoca {} de {}, training loss = {}".format(e+1,num_epochs,perdidas[-1]))
            print("--------------------------------------------")

    return perdidas

In [19]:
# Evaluate
from sklearn.metrics import accuracy_score
logsoftmax = torch.nn.LogSoftmax(dim=0)

def evaluate(loader, model, verbose=True):
    y_pred, y_true = [], []
    model.eval()
    
    with torch.no_grad():
        for i, (data, labels) in enumerate(loader):
            output = model(data)
            y_pred.append(torch.argmax(logsoftmax(output),axis=1).detach().numpy())

            y_true.append(labels.detach().numpy())
    
    score =  accuracy_score(np.concatenate(y_pred), np.concatenate(y_true))

    if verbose:
        print("Accuracy del modelo {}: {}".format(model.__class__.__name__,score))
    
    return score

A modo de ejemplo incluimos el accuracy de el modelo feed-forwards sin entrenar

In [20]:
evaluate(test_loader,model_fc)

Accuracy del modelo FCClassifier: 0.09521117204726635


0.09521117204726635

In [27]:
evaluate(test_loader,model_CNN1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

## 1.7 Comparación de Resultados

<p align="center">
  <img src="https://media2.giphy.com/media/icJA0VF7ntoEL18Jez/giphy.gif"  width="200">
</p>

Construidas las funciones de entrenamiento y evaluación, entrene a las redes que construyo y compare los resultados obtenidos con todas las redes señalando cual posee mejor rendimiento. Comente una diferencia entre las redes Fully Connected y CNN podría generar un mejor desempeño en una u otra en la tarea de clasificación de imágenes.

- [ ] Entrenar las redes.
- [ ] Evaluar las redes.
- [ ] Comentar los resultados obtenidos y visualizar si existe una diferencia significativa en el rendimiento debido a la naturaleza de la red.

In [21]:
perdidas = {}

In [22]:
%%time
print("Entrenamiento de modelo FCClassifier")
perdidas["FCClassifier"] = train(model_fc,train_loader,n_epochs)

Entrenamiento de modelo FCClassifier
epoca 1 de 5, training loss = 0.2586401402950287
--------------------------------------------
epoca 2 de 5, training loss = 0.023460598662495613
--------------------------------------------
epoca 3 de 5, training loss = 0.03540148213505745
--------------------------------------------
epoca 4 de 5, training loss = 0.0019388062646612525
--------------------------------------------
epoca 5 de 5, training loss = 0.006907980423420668
--------------------------------------------
CPU times: user 9min 6s, sys: 17.3 s, total: 9min 23s
Wall time: 1min 32s


In [ ]:
%%time
print("Entrenamiento de modelo CNNClassifier1")
perdidas["CNNClassifier1"] = train(model_CNN1,train_loader,n_epochs)

In [ ]:
%%time
print("Entrenamiento de modelo CNNClassifier2")
perdidas["CNNClassifier2"] = train(model_CNN2,train_loader,n_epochs)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

y_tag = [model.__class__.__name__ for model in modelos]

sns.set_theme(style="whitegrid")
_, ax = plt.subplots(figsize=(12,5))

for model in y_tag:
    sns.lineplot(data=perdidas[model], ax=ax, markers=['o','o','o'])
ax.legend(y_tag)
ax.set(xlabel='iteración',ylabel='error',title='Error versus iteraciones para distintos modelos')
plt.show()

In [26]:
for model in modelos:
    evaluate(test_loader,model)

Accuracy del modelo FCClassifier: 0.8956860971334879


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

> Comente los resultados

# Conclusión

Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/vKSR-ZakVMIAAAAC/pochitadancing-pochita.gif">
</p>